In [ ]:
# script that uses an excel table of CAS numbers of a couple of chemical elements as an input
# the table will be selected with tkinter
# afterwards the pubchem (a chemical database) will be searched for every element and put in a new column
# finally it extracts the source URL of the GHS classification of the European Chemical Agency (ECHA) of each element and puts the URL in a new column

import numpy as np
import pandas as pd
import requests
import tkinter
from tkinter import *
from tkinter import filedialog
import os
from thermo.chemical import Chemical
from tqdm import tqdm
import time

# open dialog to load the excel spreadsheet
root = tkinter.Tk()
request_load_file = filedialog.askopenfile(initialdir=os.getcwd(), filetypes=[('Excel Files', '*.xlsx'), ('CSV Files', '*.csv')])
if request_load_file:
    filepath = os.path.abspath(request_load_file.name)
root.destroy()

if filepath.endswith('.xlsx'):
    df_inventory = pd.read_excel(filepath)
elif filepath.endswith('.csv'):
    df_inventory = pd.read_csv(filepath)

# create new columns
df_inventory['PubChem ID'] = np.nan
df_inventory['ECHA URL'] = np.nan
df_inventory['GHS'] = np.nan
df_inventory['H Statements'] = np.nan
df_inventory['Signal'] = np.nan


# resolve PubChem IDs from CAS numbers
for i in df_inventory['CAS']:
    try:
        chem = Chemical(f'{i}')
        df_inventory.loc[df_inventory['CAS'] == i, 'PubChem ID'] = chem.PubChem
    except Exception:
        pass

from tqdm import tqdm

# Progress Part One: extract ECHA URL from PubChem data
pubchem_ids = set(df_inventory['PubChem ID'].dropna())

for chem_id in tqdm(pubchem_ids, desc="Progress Part One: Extracting ECHA URLs"):
    try:
        result = requests.get(
            f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{int(chem_id)}/JSON/?response_type=display&heading=GHS%20Classification'
        )
        data = result.json()
        sections = data['Record']['Section']

        def find_echa_url(sections):
            for section in sections:
                if section.get('TOCHeading') == 'GHS Classification':
                    for info in section.get('Information', []):
                        if info.get('Name') == 'ECHA C&L Notifications Summary':
                            if 'Value' in info:
                                for item in info['Value'].get('StringWithMarkup', []):
                                    for markup in item.get('Markup', []):
                                        if 'URL' in markup:
                                            return markup['URL']
                if 'Section' in section:
                    url = find_echa_url(section['Section'])
                    if url:
                        return url
            return None

        echa_url = find_echa_url(sections)
        if echa_url:
            df_inventory.loc[df_inventory['PubChem ID'] == chem_id, 'ECHA URL'] = echa_url

    except Exception:
        pass

In [ ]:
# second part of the script
# visits every ECHA source URL and extracts the information of the summary table of the GHS classification (always named "table 3")
# extracts GHS pictograms classification (e. g. GHS01 explosive) and H statements (e. g. H360	May damage fertility or the unborn child) and signal word (danger or warning)
# if summary table 3 does not exist, skips the element

def retrieve_echa_data(df):
    #session setup with user-agent setting
    session = requests.Session()
    session.headers.update({
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/122.0.0.0 Safari/537.36",
        "Referer": "https://www.google.com"
    })
    
    time.sleep(3)

    urls_to_process = df[df['ECHA URL'].notna()]

    #main loop with tqdm to track progress
    for idx, row in tqdm(urls_to_process.iterrows(), total=len(urls_to_process), desc="Retrieving ECHA data"):
        url = row['ECHA URL']
        try:
            response = session.get(url)
            if response.status_code != 200:
                print(f"Failed to fetch the page at {url}. Status code: {response.status_code}")
                continue

            #html parsing of website
            soup = BeautifulSoup(response.content, 'html.parser')

            #search for table 3
            table_heading = soup.find('span', string=re.compile(r'CLP Classification \(Table 3\)'))
            if not table_heading:
                print(f"Table 3 not found for {url}")
                continue

            table = table_heading.find_next('table')
            if not table:
                print(f"Classification table not found for {url}")
                continue

            h_statements = set()
            ghs_codes = set()
            signal_word = None

            #extract H statements and signal words
            for tr in table.find_all('tr', class_='results-row'):
                tds = tr.find_all('td')
                for td in tds:
                    h_spans = td.find_all('span', class_='CLInventoryHelpCursor')
                    for span in h_spans:
                        text = span.get_text(strip=True)
                        if re.match(r'H\d{3}', text):
                            h_statements.add(text)

                    text_content = td.get_text(" ", strip=True)
                    ghs_codes.update(re.findall(r'GHS\d{2}', text_content))
                    if 'Dgr' in text_content:
                        signal_word = 'Danger'
                    elif 'Wng' in text_content:
                        signal_word = 'Warning'

            # Save extracted data
            df.at[idx, 'H Statements'] = ', '.join(sorted(h_statements)) if h_statements else None
            df.at[idx, 'GHS'] = ', '.join(sorted(ghs_codes)) if ghs_codes else None
            df.at[idx, 'Signal'] = signal_word

        except Exception as e:
            print(f"Error retrieving data for URL {url}: {e}")



retrieve_echa_data (df_inventory)

# save output
df_inventory.to_excel(filepath[:filepath.rfind('.')] + '_echa_dat.xlsx')